In [14]:
import numpy as np
import tensorflow as tf
import codecs
from os import listdir
import nltk
from gensim.models import Word2Vec
from random import randint
import data

In [15]:
vocab_size = 1681
d, id_to_token = data.get_data(vocab_size)
for k in id_to_token:
    if id_to_token[k] == 'eos':
        id_to_token[k] = '\n'
print len(d)
x, y = d[10:20], d[11:21]
print [np.argmax(j) for j in x]
print [np.argmax(j) for j in y]

[u'i', u'would', u'like', u'to', u'talk', u'today', u'about', u'how', u'to', u'develop', u'a', u'new', u'foreign', u'policy', u'direction', u'for', u'our', u'country', u'one', u'that', u'replaces', u'randomness', u'with', u'purpose', u'ideology', u'with', u'strategy', u'and', u'chaos', u'with', u'peace', u'it', u'is', u'time', u'to', u'shake', u'the', u'rust', u'off', u'of']
1681
10229
[5, 124, 63, 70, 371, 19, 13, 40, 57, 8]
[124, 63, 70, 371, 19, 13, 40, 57, 8, 1437]


In [16]:
# dirrs = ['sentiment/train/', 'sentiment/test/']
# sent = []
# for dirr in dirrs:
#     print dirr
#     l = listdir(dirr+'pos')
#     print 'pos'
#     for r in l:
#         t = codecs.open(dirr+'pos/'+r,'r',encoding='utf8').read()
#         sent.append(nltk.word_tokenize(t))
#     l = listdir(dirr+'neg')
#     print 'neg'
#     for r in l:
#         t = codecs.open(dirr+'neg/'+r,'r',encoding='utf8').read()
#         sent.append(nltk.word_tokenize(t))

In [17]:
# m = Word2Vec.load('embedding.model')
# data = []
# for s in sent:
#     a = [m[x] for x in s]
#     data.append(a)
# data = np.array(data)

In [18]:
# def sam(d):
#     r = randint(0, data.shape[0] - 1)
#     t = d[r]
#     x = t[:-1]
#     y = t[1:]
#     return np.array(x), np.array(y)

# s = sample(data)
# s[0].shape, s[1].shape

In [19]:
# hyper-parameters
epochs = 10000
hidden_layer = 128
inp_out_size = 1681
learning_rate = 0.1
num_steps = 15

In [20]:
tf.reset_default_graph()
# Wxh = tf.Variable([[.1, .2], [.3, .4], [.5, .6], [.7, .8]], dtype=tf.float32)
# Whh = tf.Variable([[.1, .2], [.3, .4]], dtype=tf.float32)
# Why = tf.Variable([[.1, .2, .3, .4], [.4, .5, .6, .7]], dtype=tf.float32)
Wxh = tf.Variable(tf.random_normal([inp_out_size, hidden_layer], mean=0, stddev=0.001))
Whh = tf.Variable(tf.random_normal([hidden_layer, hidden_layer], mean=0, stddev=0.001))
Why = tf.Variable(tf.random_normal([hidden_layer, inp_out_size], mean=0, stddev=0.001))
bh = tf.Variable(tf.zeros([hidden_layer]), dtype=tf.float32)
by = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)
print tf.trainable_variables()

[<tensorflow.python.ops.variables.Variable object at 0x7f811731e150>, <tensorflow.python.ops.variables.Variable object at 0x7f81163f6bd0>, <tensorflow.python.ops.variables.Variable object at 0x7f8106f0f910>, <tensorflow.python.ops.variables.Variable object at 0x7f8106f1ded0>, <tensorflow.python.ops.variables.Variable object at 0x7f8106f0fa10>]


In [21]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    h = tf.tanh((tf.matmul(p, Whh))  + tf.matmul(i, Wxh) + bh)
    h = tf.reshape(h, [hidden_layer])
    return h

In [22]:
a = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
b = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
initial = tf.placeholder(shape=[hidden_layer], dtype=tf.float32)
states = tf.scan(recurrence, a, initializer=initial)
outputs = tf.nn.softmax(tf.matmul(states, Why) + by)
loss = -tf.reduce_mean(b*tf.log(outputs))
# loss = tf.sqrt(tf.reduce_sum(tf.square(tf.sub(outputs, b))))
optimizer = tf.train.AdagradOptimizer(learning_rate)

# clipping gradients between -1 and 1.
grad_var_pairs = optimizer.compute_gradients(loss, tf.trainable_variables())
clipped_grad_var_pairs = [(tf.clip_by_value(gv[0], -1, 1), gv[1]) for gv in grad_var_pairs]
optimize_op = optimizer.apply_gradients(clipped_grad_var_pairs)

# optimize_op = optimizer.minimize(loss)

In [23]:
def generate(sess, n):
    x, _ = data.sample(d, 1)
    gen = [id_to_token[np.argmax(x[0])]]
    h = np.zeros(hidden_layer)
    for i in range(n):
        o, h = sess.run([outputs, states], {a:x, initial: h})
        h = h.reshape(hidden_layer)
        o = np.argmax(o[0])
        gen.append(id_to_token[o])
        x = [0] * inp_out_size
        x[o] = 1
#         print np.argmax(x)
        x = [x]
    print ' '.join(gen)

In [24]:
init = tf.initialize_all_variables()
saver = tf.train.Saver()
sess = tf.Session()
ix = 0
smooth_loss = -np.log(1.0 / vocab_size) * num_steps

In [25]:
# if already trained previously, just restore
to_restore = False

if to_restore:
    saver.restore(sess, 'model.ckpt')
else:
    sess.run(init)

In [26]:
print epochs
h = np.zeros(hidden_layer)
for i in range(epochs):
#     x, y = data.sample(d, num_steps)
    if ix + num_steps >= len(d):
        ix = 0
    num_steps = num_steps - 1 if i % 2 == 0 else num_steps + 1 # variable length sequences
    h = np.zeros(hidden_layer)
    x, y = d[ix : ix + num_steps], d[ix + 1 : ix + num_steps + 1]   
    l, o, _ = sess.run([loss, outputs, optimize_op], {a: x, b: y, initial: h}) 
    smooth_loss = smooth_loss * 0.999 + l * 0.001
    if i % 1000 == 0:
        print('epoch {0}, loss = {1}'.format(i, l))
    ix += num_steps

10000
epoch 0, loss = 0.00441828789189


KeyboardInterrupt: 

In [ ]:
# saving model
saver.save(sess, 'model.ckpt')

In [ ]:
generate(sess, 500)

In [ ]:
sess.run(Whh)